# House price classifier

Predict whether house price is above median based on house and neighborhood characteristics

In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)


Populating the interactive namespace from numpy and matplotlib


## Data preparation

Add a new feature, and split train-test. Create a validation set.

In [2]:
def clean_data(data, is_train_data=True):
    data['TotalSF'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF']
    if is_train_data == True:
        data = data[data.GrLivArea < 4000]        
    return data

In [3]:
df = pd.read_csv('data/ames/train.csv')
categorical_df = df.select_dtypes('object')
numerical_df = df.select_dtypes('number')
median_price = df.SalePrice.median()
y = df.SalePrice > median_price
numerical_df.drop(['Id', 'SalePrice'], axis=1, inplace=True)
X = pd.concat([numerical_df, pd.get_dummies(categorical_df, drop_first=True)], axis=1)
X = X.fillna(X.median()).astype('int64')

/Users/mehdi/Documents/python-client/venv/lib/python3.7/site-packages/pandas/core/frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=56)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=56)

# ML Experiment

Run with different hyperparams and pick the best.

In [5]:
import itertools
import operator
import xgboost as xgb

In [6]:
depths = [3, 5, 7]
n_estimators = [5, 10, 20]
gammas= [0, 0.1, 0.3]

In [7]:
# performance_dict = dict()
# for max_depth, n_estimator, gamma in list(itertools.product(depths, n_estimators, gammas)):
#     booster_model = xgb.XGBClassifier(max_depth=max_depth, n_estimator=n_estimator, gamma=gamma)
#     booster_model.fit(X_train, y_train)
#     print(f'Finished training model with {max_depth}, {n_estimator}, {gamma} params')
#     performance_dict.update({
#         (max_depth, n_estimator, gamma): booster_model.score(X_val, y_val)
#     })

Retrain the model with the best params.

In [8]:
best_max_depth, best_n_estimator, best_gamma = 3, 20, 0
# max(performance_dict.items(), key=operator.itemgetter(1))[0]
print(f'Best params are {best_max_depth} max_depth, {best_n_estimator} n_estimators, {best_gamma} gamma.')
best_model = xgb.XGBClassifier(
    max_depth=best_max_depth,
    n_estimator=best_n_estimator,
    gamma=best_gamma)
best_model.fit(X_train.values, y_train)

Best params are 3 max_depth, 20 n_estimators, 0 gamma.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimator=20,
              n_estimators=100, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [9]:
FI = dict(zip(X_train.columns, best_model.feature_importances_))

In [10]:
top_20_features = dict(sorted(FI.items(), key=operator.itemgetter(1), reverse=True)[:20])

In [11]:
mini_X_train = X_train[top_20_features.keys()]
mini_X_test = X_test[top_20_features.keys()]
print(mini_X_train.shape, mini_X_test.shape)

(817, 20) (438, 20)


In [12]:
mini_model = xgb.XGBClassifier(
    max_depth=best_max_depth,
    n_estimator=best_n_estimator,
    gamma=best_gamma)
mini_model.fit(mini_X_train.values, y_train)
accuracy_score = mini_model.score(mini_X_test.values, y_test)
FI = dict(zip(mini_X_train.columns, mini_model.feature_importances_))

In [13]:
feature_map = {
    'YearBuilt': '3a7a81e2e6da11e985bd000000000002',
    'FullBath': '3a74509ae6da11e9bf72000000000002',
    'OverallQual': '225a218ce55911e9a244000000000002',
    'GarageFinish_Unf': '2d3d2abee55911e99a5e000000000002',
    'GarageType_Detchd': '3402dac0e55911e9a297000000000002',
    'Fireplaces': '4d2ff40ae55911e98624000000000002',
    'BsmtQual_Gd': '58bf3324e55911e99c0f000000000002',
    'KitchenQual_TA': '98c36c86e55911e99cf1000000000002',
    'GrLivArea': 'fa71695ee55911e9a4f0000000000002',
    'YearRemodAdd': '4f2c224ae55a11e9b148000000000002',
    'MSZoning_RM': '94d40e10e55a11e9aa80000000000002',
    '1stFlrSF': 'c6c484d8e55a11e9b9ee000000000002',
    'GarageCars': 'f23933f4e55a11e9b5ab000000000002',
    'LotArea': '3a6fdeb6e6da11e99130000000000002',
    'KitchenAbvGr': '3a78498ce6da11e9823f000000000002',
    'TotalBsmtSF': '56760bb4e55b11e99838000000000002',
    'ScreenPorch': '7f02b486e55b11e986ce000000000002',
    'MSZoning_RL': 'ba31c32ce55b11e9a3fc000000000002',
    'BsmtFinType1_GLQ': '077ab0f4e55c11e9ad3f000000000002',
    '2ndFlrSF': '1d9d5110e55c11e98f79000000000002',
    'SalePrice': '18f521d8e5fa11e9ab1b000000000002'
}
authors_ids = ['5b69b6dee6f111e9a037000000000007']
source_datasets = ['f4ded1b0dfca11e9ab5e000000000004']
neighborhoods = ['99397a46e6f111e9953200000000000a']
population = 'dd103c38ec6411e99cf6000000000008'

# Submission

In [14]:
experiment_name = "Ames, Iowa housing price classifier using top 20 most performant features."
experiment_lede = "XBoost model to predict whether a house price is above median."
experiment_description = """
In this experiment, an XGBoost model was trained all the features found in Kaggle's Ames, Iowa housing price dataset.
The top 20 most performat features was selected with Gain as the feature contribution method.
"""

In [15]:
import system_client as system
from onnxmltools.convert.common.data_types import Int64TensorType
num_features = mini_X_test.shape[1]
initial_type = [('feature_input', Int64TensorType([1, num_features]))]

In [18]:
with system.SupervisedMLSession(config_path='local.json', authors=authors_ids) as session1:
    evaluation = session1.add_evaluation(mini_model, mini_X_test, pd.DataFrame(y_test), initial_types=initial_type)
    system_model = session1.add_model(
        label='Best model',
        algorithm='XGBoost',
        performance={'AccuracyScore': accuracy_score},
        target_name='SalePrice',
        model_params=best_model.get_params(),
        source_datasets=source_datasets,
        model_population=population,
        neighborhoods=neighborhoods,
        evaluation=evaluation
    )
    session1.map_features_to_variables(feature_to_variable_map=feature_map)
    session1.add_experiment_metadata(
        name=experiment_name,
        lede=experiment_lede,
        description=experiment_description
    )
#     session1.submit()

The maximum opset needed by this model is only 1.


Setting Mehdi Jamei as an author.
EVALUATOR OUTPUT 2019-10-14 16-31-49 Loading the ONNX Model...EVALUATOR OUTPUT 2019-10-14 16-31-49 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-31-49 Creating descriptive stats for X...EVALUATOR OUTPUT 2019-10-14 16-31-50 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-31-50 Creating descriptive stats for y...EVALUATOR OUTPUT 2019-10-14 16-31-51 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-31-51 Calculating model F1Score...EVALUATOR OUTPUT 2019-10-14 16-31-51 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-31-51 Calculating model AccuracyScore...EVALUATOR OUTPUT 2019-10-14 16-31-51 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-31-51 Calculating model RocAucScore...EVALUATOR OUTPUT 2019-10-14 16-31-51 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-31-51 Calculating model RocCurve...EVALUATOR OUTPUT 2019-10-14 16-31-51 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-31-51 Calculating model ConfusionMatrix...EVALUATOR OUTPUT 2019-10-14 16-31-51 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-3

In [24]:
session1.submit()

https://localhost:4200/write/subgraph/ce2b2bdb616617b007fdb192febb6fd86a5cfaddef43eceec4ea9f0b37b478dd


{'subgraph_id': 'ce2b2bdb616617b007fdb192febb6fd86a5cfaddef43eceec4ea9f0b37b478dd',
 'upload_urls': {}}

# Multiple hyperparams

In [19]:
experiment2_name = "Ames, Iowa housing price classifier with various hyperparameters."
experiment2_lede = "XBoost model to predict whether a house price is above median."
experiment2_description = """
In this experiment, an XGBoost model was trained all the features found in Kaggle's Ames, Iowa housing price dataset.
The top 20 most performat features was selected with Gain as the feature contribution method.
Same model was trained and tested with different hyper parameters.
"""

In [21]:
with system.SupervisedMLSession(config_path='local.json', authors=authors_ids) as session:
 
    for max_depth, n_estimator, gamma in [
        (3, 15, 0),
        (3, 20, 0),
        (4, 15, 0),
        (2, 15, 0),
        (3, 15, 0.01)
    ]:
        model = xgb.XGBClassifier(
            max_depth=max_depth,
            n_estimator=n_estimator,
            gamma=gamma)
        model.fit(mini_X_train.values, y_train)
        evaluation = session.add_evaluation(
            model,
            mini_X_test,
            pd.DataFrame(y_test),
            initial_types=initial_type)
        system_model = session.add_model(
            target_name='SalePrice',
            algorithm='XGBoost',
            label=f'max_depth={max_depth}, n_estimator={n_estimator}, gamma={gamma}',
            model_params=model.get_params(),
            source_datasets=source_datasets,
            model_population=population,
            evaluation=evaluation,
            neighborhoods=neighborhoods)
    session.map_features_to_variables(feature_to_variable_map=feature_map)
    session.add_experiment_metadata(
        name=experiment2_name,
        lede=experiment2_lede,
        description=experiment2_description
    )
    session.submit()

Setting Mehdi Jamei as an author.


The maximum opset needed by this model is only 1.


EVALUATOR OUTPUT 2019-10-14 16-46-38 Loading the ONNX Model...EVALUATOR OUTPUT 2019-10-14 16-46-39 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-46-39 Creating descriptive stats for X...EVALUATOR OUTPUT 2019-10-14 16-46-39 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-46-39 Creating descriptive stats for y...EVALUATOR OUTPUT 2019-10-14 16-46-39 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-46-40 Calculating model F1Score...EVALUATOR OUTPUT 2019-10-14 16-46-40 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-46-40 Calculating model AccuracyScore...EVALUATOR OUTPUT 2019-10-14 16-46-40 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-46-40 Calculating model RocAucScore...EVALUATOR OUTPUT 2019-10-14 16-46-40 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-46-40 Calculating model RocCurve...EVALUATOR OUTPUT 2019-10-14 16-46-40 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-46-40 Calculating model ConfusionMatrix...EVALUATOR OUTPUT 2019-10-14 16-46-40 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-46-40 Calculating permutation for F

The maximum opset needed by this model is only 1.


EVALUATOR OUTPUT 2019-10-14 16-47-09 Loading the ONNX Model...EVALUATOR OUTPUT 2019-10-14 16-47-09 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-09 Creating descriptive stats for X...EVALUATOR OUTPUT 2019-10-14 16-47-10 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-10 Creating descriptive stats for y...EVALUATOR OUTPUT 2019-10-14 16-47-10 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-10 Calculating model F1Score...EVALUATOR OUTPUT 2019-10-14 16-47-10 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-10 Calculating model AccuracyScore...EVALUATOR OUTPUT 2019-10-14 16-47-10 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-10 Calculating model RocAucScore...EVALUATOR OUTPUT 2019-10-14 16-47-10 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-10 Calculating model RocCurve...EVALUATOR OUTPUT 2019-10-14 16-47-10 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-10 Calculating model ConfusionMatrix...EVALUATOR OUTPUT 2019-10-14 16-47-10 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-10 Calculating permutation for F

...

EVALUATOR OUTPUT 2019-10-14 16-47-44 Loading the ONNX Model...EVALUATOR OUTPUT 2019-10-14 16-47-45 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-45 Creating descriptive stats for X...EVALUATOR OUTPUT 2019-10-14 16-47-46 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-46 Creating descriptive stats for y...EVALUATOR OUTPUT 2019-10-14 16-47-46 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-46 Calculating model F1Score...EVALUATOR OUTPUT 2019-10-14 16-47-46 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-46 Calculating model AccuracyScore...EVALUATOR OUTPUT 2019-10-14 16-47-46 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-46 Calculating model RocAucScore...EVALUATOR OUTPUT 2019-10-14 16-47-46 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-46 Calculating model RocCurve...EVALUATOR OUTPUT 2019-10-14 16-47-46 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-46 Calculating model ConfusionMatrix...EVALUATOR OUTPUT 2019-10-14 16-47-46 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-47-46 Calculating permutation for F

The maximum opset needed by this model is only 1.


EVALUATOR OUTPUT 2019-10-14 16-48-24 Loading the ONNX Model...EVALUATOR OUTPUT 2019-10-14 16-48-24 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-48-24 Creating descriptive stats for X...EVALUATOR OUTPUT 2019-10-14 16-48-25 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-48-25 Creating descriptive stats for y...EVALUATOR OUTPUT 2019-10-14 16-48-25 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-48-25 Calculating model F1Score...EVALUATOR OUTPUT 2019-10-14 16-48-25 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-48-25 Calculating model AccuracyScore...EVALUATOR OUTPUT 2019-10-14 16-48-25 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-48-25 Calculating model RocAucScore...EVALUATOR OUTPUT 2019-10-14 16-48-25 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-48-25 Calculating model RocCurve...EVALUATOR OUTPUT 2019-10-14 16-48-25 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-48-25 Calculating model ConfusionMatrix...EVALUATOR OUTPUT 2019-10-14 16-48-26 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-48-26 Calculating permutation for F

The maximum opset needed by this model is only 1.


EVALUATOR OUTPUT 2019-10-14 16-49-07 Loading the ONNX Model...EVALUATOR OUTPUT 2019-10-14 16-49-07 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-49-07 Creating descriptive stats for X...EVALUATOR OUTPUT 2019-10-14 16-49-09 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-49-09 Creating descriptive stats for y...EVALUATOR OUTPUT 2019-10-14 16-49-09 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-49-09 Calculating model F1Score...EVALUATOR OUTPUT 2019-10-14 16-49-09 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-49-09 Calculating model AccuracyScore...EVALUATOR OUTPUT 2019-10-14 16-49-10 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-49-10 Calculating model RocAucScore...EVALUATOR OUTPUT 2019-10-14 16-49-10 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-49-10 Calculating model RocCurve...EVALUATOR OUTPUT 2019-10-14 16-49-10 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-49-10 Calculating model ConfusionMatrix...EVALUATOR OUTPUT 2019-10-14 16-49-10 SUCCESS...
EVALUATOR OUTPUT 2019-10-14 16-49-10 Calculating permutation for F

# Different Algorithms

In [22]:
experiment3_name = "Ames, Iowa housing price classifier with various classification algorithms."
experiment3_lede = "XBoost model to predict whether a house price is above median."
experiment3_description = """
In this experiment, an XGBoost model was trained all the features found in Kaggle's Ames, Iowa housing price dataset.
The top 20 most performat features was selected with Gain as the feature contribution method.
Random Forest, Logistic Regression and XGBoost models were trained and tested.
"""

In [23]:
import sklearn

In [24]:
rf_model = sklearn.ensemble.RandomForestClassifier()
logit_model = sklearn.linear_model.LogisticRegression()
xgb_model = xgb.XGBClassifier(
            max_depth=best_max_depth,
            n_estimator=best_n_estimator,
            gamma=best_gamma)

In [18]:
with system.SupervisedMLSession(config_path='local.json', authors=authors_ids) as session3:
 
    for model in [xgb_model, rf_model, logit_model]:
        model.fit(mini_X_train.values, y_train.astype(int))
        evaluation = session3.add_evaluation(
            model,
            mini_X_test,
            pd.DataFrame(y_test.astype(int)),
            initial_types=initial_type)
        system_model = session3.add_model(
            target_name='SalePrice',
            algorithm=model.__class__.__name__,
            label=f'algorithm={model.__class__.__name__}',
            model_params=model.get_params(),
            source_datasets=source_datasets,
            model_population=population,
            evaluation=evaluation,
            neighborhoods=neighborhoods)
    session3.map_features_to_variables(feature_to_variable_map=feature_map)
    session3.add_experiment_metadata(
        name=experiment3_name,
        lede=experiment3_lede,
        description=experiment3_description
    )
    session3.submit()

Setting Mehdi Jamei as an author.


The maximum opset needed by this model is only 1.


EVALUATOR OUTPUT 2019-10-09 10-55-25 Loading the ONNX Model...EVALUATOR OUTPUT 2019-10-09 10-55-25 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-55-25 Creating descriptive stats for X...EVALUATOR OUTPUT 2019-10-09 10-55-27 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-55-27 Creating descriptive stats for y...EVALUATOR OUTPUT 2019-10-09 10-55-27 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-55-27 Calculating model F1Score...EVALUATOR OUTPUT 2019-10-09 10-55-27 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-55-27 Calculating model AccuracyScore...EVALUATOR OUTPUT 2019-10-09 10-55-27 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-55-27 Calculating model RocAucScore...EVALUATOR OUTPUT 2019-10-09 10-55-28 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-55-28 Calculating model RocCurve...EVALUATOR OUTPUT 2019-10-09 10-55-28 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-55-28 Calculating model ConfusionMatrix...EVALUATOR OUTPUT 2019-10-09 10-55-28 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-55-28 Calculating permutation for F

The maximum opset needed by this model is only 9.
The maximum opset needed by this model is only 1.


EVALUATOR OUTPUT 2019-10-09 10-56-49 Loading the ONNX Model...EVALUATOR OUTPUT 2019-10-09 10-56-50 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-56-50 Creating descriptive stats for X...EVALUATOR OUTPUT 2019-10-09 10-56-51 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-56-51 Creating descriptive stats for y...EVALUATOR OUTPUT 2019-10-09 10-56-52 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-56-52 Calculating model F1Score...EVALUATOR OUTPUT 2019-10-09 10-56-52 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-56-52 Calculating model AccuracyScore...EVALUATOR OUTPUT 2019-10-09 10-56-52 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-56-52 Calculating model RocAucScore...EVALUATOR OUTPUT 2019-10-09 10-56-52 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-56-52 Calculating model RocCurve...EVALUATOR OUTPUT 2019-10-09 10-56-52 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-56-52 Calculating model ConfusionMatrix...EVALUATOR OUTPUT 2019-10-09 10-56-54 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-56-54 Calculating permutation for F

The maximum opset needed by this model is only 9.
The maximum opset needed by this model is only 1.


EVALUATOR OUTPUT 2019-10-09 10-57-30 Loading the ONNX Model...EVALUATOR OUTPUT 2019-10-09 10-57-30 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-57-30 Creating descriptive stats for X...EVALUATOR OUTPUT 2019-10-09 10-57-32 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-57-32 Creating descriptive stats for y...EVALUATOR OUTPUT 2019-10-09 10-57-32 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-57-32 Calculating model F1Score...EVALUATOR OUTPUT 2019-10-09 10-57-32 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-57-32 Calculating model AccuracyScore...EVALUATOR OUTPUT 2019-10-09 10-57-32 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-57-32 Calculating model RocAucScore...EVALUATOR OUTPUT 2019-10-09 10-57-32 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-57-32 Calculating model RocCurve...EVALUATOR OUTPUT 2019-10-09 10-57-32 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-57-32 Calculating model ConfusionMatrix...EVALUATOR OUTPUT 2019-10-09 10-57-32 SUCCESS...
EVALUATOR OUTPUT 2019-10-09 10-57-32 Calculating permutation for F